In [ ]:
!pip install requests beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import re
from bs4 import BeautifulSoup

url_base = "https://www.vinmec.com/vi/danh-sach/bac-si/ca-nuoc/noitieng-tieng-viet?page={}"
doctor_info = []

# loop through each page (page 1 to 10) and extract information for each doctor
for page_number in range(1, 11):
    url = url_base.format(page_number)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    doctors = soup.find_all("li", class_="")

    # loop through each doctor and extract their information
    for doctor in doctors:
        name_element = doctor.find("h2")
        name = name_element.text.strip() if name_element is not None else ""

        if name != '':
            brief_section = doctor.find("dl", class_="brief")
            if brief_section is not None:
                brief = []
                dt_elements = brief_section.find_all("dt")
                dd_elements = brief_section.find_all("dd")
                for dt, dd in zip(dt_elements, dd_elements):
                    dt_text = dt.text.strip() if dt is not None else ""
                    dd_text = dd.text.strip() if dd is not None else ""
                    brief.append((dt_text, dd_text))

                rank = brief[0][1].replace('\n', '').replace('\t', '')
                majorityFull = brief[1][1].replace('\n', '').replace('\t', '')
                desc = doctor.find("div", class_="desc").text.strip() if doctor.find("div", class_="desc") else ""
                desc = desc.replace('\n', '').replace('\t', '').replace('...Xem thêm', '...')

            media_section = doctor.find("div", class_="media")
            if media_section is not None:
                profile_image_element = media_section.find("a")
                if 'style' in profile_image_element.attrs:
                  profile_image_style = profile_image_element['style']
                  match = re.search(r'url\((.*?)\)', profile_image_style)
                  if match:
                      profile_image = match.group(1)
                  else:
                      profile_image = ""
            else:
                profile_image_style = ""
                profile_image = ""

            doctor_info.append({
                "name": name,
                "avaImage": profile_image,
                "rank": rank,
                "majorityFull": majorityFull,
                "desc": desc
            })

# print information for each doctor
for doctor in doctor_info:
    print(doctor)


{'name': 'Bác sĩ Nguyễn Thanh Liêm', 'avaImage': 'https://vinmec-prod.s3.amazonaws.com/images/12_01_2019_11_08_53_877612.jpeg', 'rank': 'Giáo sư,Tiến sĩ, Bác sĩ', 'majorityFull': 'Nhi,Y học tái tạo (Tự kỷ bại não, Tế bào gốc),Nghiên cứu khoa học', 'desc': 'GS. TS. Nguyễn Thanh Liêm có hơn 50 năm kinh nghiệm trong lĩnh vực Ngọai Nhi và là người tiên phong đưa các liệu pháp trong lĩnh vực Y học tái tạo và trị liệu...'}
{'name': 'Bác sĩ Phạm Nhật An', 'avaImage': 'https://vinmec-prod.s3.amazonaws.com/images/05_01_2019_03_03_31_282527.jpeg', 'rank': 'Giáo sư,Tiến sĩ, Bác sĩ', 'majorityFull': 'Nhi', 'desc': 'GS.TS.BS Phạm Nhật An có 45 năm kinh nghiệm trong lĩnh vực Nhi khoa. Năm 2016, BS Nhật An được phong hàm Giáo sư y học. Ông đã tham gia các khoá học chuyên sâu...'}
{'name': 'Dược sĩ Phan Quỳnh Lan', 'avaImage': 'https://vinmec-prod.s3.amazonaws.com/images/2015-11-19_171242.6189150000.jpeg', 'rank': 'Tiến sĩ', 'majorityFull': 'Dược', 'desc': 'TS. DS Phan Quỳnh Lan có hơn 15 năm kinh ngh

In [ ]:
print(len(doctor_info))

100


In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 19.1 MB/s eta 0:00:00


In [ ]:
import random
from datetime import date, timedelta
from unidecode import unidecode

majority_list = [    {"majority": "general", "majorityFull": "Sức khỏe tổng quát"},    {"majority": "cardio", "majorityFull": "Tim mạch"},    {"majority": "oncology", "majorityFull": "Ung bướu"},    {"majority": "obgyn", "majorityFull": "Sản phụ khoa và hỗ trợ sinh sản"},    {"majority": "pediatrics", "majorityFull": "Nhi"},    {"majority": "gastro", "majorityFull": "Tiêu hóa - Gan mật"},    {"majority": "ortho", "majorityFull": "Cơ xương khớp"},    {"majority": "tchm", "majorityFull": "Y học cổ truyền"}]

def generate_payload(doctor):
    doctorname = "".join([name[0] for name in doctor["name"].split()[:-1]]).lower() + doctor["name"].split()[-1].lower()
    doctorname = unidecode(doctorname)
    email = doctorname + "@bkhos.com"
    password = "123"
    fullname = doctor["name"]
    gender = random.choice(["male", "female"])
    dob = (date.today() - timedelta(days=random.randint(365 * 18, 365 * 48))).strftime("%Y-%m-%d")
    rank = doctor["rank"]
    majority_dict = random.choice(majority_list)
    majority = majority_dict["majority"]
    majorityFull = majority_dict["majorityFull"]
    workday = random.sample(range(1, 8), random.randint(3, 5))
    desc = doctor["desc"]
    avaImage = doctor["avaImage"]

    payload = {
        "doctorname": doctorname,
        "password": password,
        "email": email,
        "fullname": fullname,
        "gender": gender,
        "dob": dob,
        "rank": rank,
        "majority": majority,
        "majorityFull": majorityFull,
        "workday": workday,
        "desc": desc,
        "avaImage": avaImage
    }

    return payload

# generate payload for each doctor in doctor_info
payload_list = [generate_payload(doctor) for doctor in doctor_info]

print(payload_list)


[{'doctorname': 'bsntliem', 'password': '123', 'email': 'bsntliem@bkhos.com', 'fullname': 'Bác sĩ Nguyễn Thanh Liêm', 'gender': 'female', 'dob': '1985-06-19', 'rank': 'Giáo sư,Tiến sĩ, Bác sĩ', 'majority': 'pediatrics', 'majorityFull': 'Nhi', 'workday': [1, 6, 4, 7, 5], 'desc': 'GS. TS. Nguyễn Thanh Liêm có hơn 50 năm kinh nghiệm trong lĩnh vực Ngọai Nhi và là người tiên phong đưa các liệu pháp trong lĩnh vực Y học tái tạo và trị liệu...', 'avaImage': 'https://vinmec-prod.s3.amazonaws.com/images/12_01_2019_11_08_53_877612.jpeg'}, {'doctorname': 'bspnan', 'password': '123', 'email': 'bspnan@bkhos.com', 'fullname': 'Bác sĩ Phạm Nhật An', 'gender': 'male', 'dob': '1989-07-12', 'rank': 'Giáo sư,Tiến sĩ, Bác sĩ', 'majority': 'general', 'majorityFull': 'Sức khỏe tổng quát', 'workday': [5, 2, 3], 'desc': 'GS.TS.BS Phạm Nhật An có 45 năm kinh nghiệm trong lĩnh vực Nhi khoa. Năm 2016, BS Nhật An được phong hàm Giáo sư y học. Ông đã tham gia các khoá học chuyên sâu...', 'avaImage': 'https://vinm

In [ ]:
import requests
import json

url = "http://localhost:5000/api/doctors/auth/register"

headers = {
    "Content-Type": "application/json"
}

for payload in payload_list:  # chỉ lấy 5 phần tử đầu
    # Format the payload data as a JSON string
    json_payload = json.dumps(payload)

    # Send a POST request to the server with the JSON payload
    response = requests.post(url, headers=headers, data=json_payload)

    # Check the response status code and content
    if response.status_code == 200:
        print(f"API call succeeded: {response.content}")
    else:
        print(f"API call failed: {response.status_code}")


ConnectionError: ignored

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def get_post_urls(url):
    post_urls = []
    for page_num in range(1, 20):
        page_url = url + f'?page={page_num}'
        response = requests.get(page_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        post_list = soup.find('div', {'class': 'post-list'})
        anchors = post_list.find_all('a', href=True)

        for a in anchors:
            img_tag = a.find('img')
            if img_tag:
                link = a['href']
                img_src = img_tag['v-lazy'] if 'v-lazy' in img_tag.attrs else None

                post_urls.append({
                    'link': link,
                    'img_src': img_src
                })

    return post_urls

def crawl_post_urls():
    base_url = 'https://www.vinmec.com/vi'
    urls = [
        '/y-hoc-co-truyen/suc-khoe-thuong-thuc/',
        '/co-xuong-khop/suc-khoe-thuong-thuc/',
        '/tieu-hoa-gan-mat/thong-tin-suc-khoe/',
        '/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/',
        '/ung-buou-xa-tri/thong-tin-suc-khoe/',
        '/tin-tuc/thong-tin-suc-khoe/nhi/',
        '/tim-mach/thong-tin-suc-khoe/',
        '/tin-tuc/thong-tin-suc-khoe/san-phu-khoa-va-ho-tro-sinh-san/'
    ]

    majorityList = [
        {"majorityCode": "general", "majorityFull": "suc-khoe-tong-quat"},
        {"majorityCode": "cardio", "majorityFull": "tim-mach"},
        {"majorityCode": "oncology", "majorityFull": "ung-buou"},
        {"majorityCode": "obgyn", "majorityFull": "san-phu-khoa-va-ho-tro-sinh-san"},
        {"majorityCode": "pediatrics", "majorityFull": "nhi"},
        {"majorityCode": "gastro", "majorityFull": "tieu-hoa-gan-mat"},
        {"majorityCode": "ortho", "majorityFull": "co-xuong-khop"},
        {"majorityCode": "tchm", "majorityFull": "y-hoc-co-truyen"}
    ]

    url_set = set()  # Set to store unique URLs

    for url in urls:
        full_url = base_url + url
        post_urls = get_post_urls(full_url)

        majority_code = None
        for majority in majorityList:
            if majority['majorityFull'] in full_url:
                majority_code = majority['majorityCode']
                break

        for post_url in post_urls:
            url_object = {
                "link": base_url + post_url['link'],
                "majority": majority_code,
                "image": post_url['img_src'],
            }
            match = re.search(r'([^/]+)/(?:$|\?)', post_url['link'])
            if match:
                url_title = match.group(1)
                url_object["url_title"] = url_title

            url_set.add(tuple(url_object.items()))

    url_list = [dict(url_tuple) for url_tuple in url_set]  # Convert tuples back to dicts

    return url_list

crawl_url = crawl_post_urls()

for url in crawl_url:
    print(url)


{'link': 'https://www.vinmec.com/vi/tieu-hoa-gan-mat/thong-tin-suc-khoe/cac-dau-hieu-bi-benh-gan/', 'majority': 'gastro', 'image': "'https://vinmec-prod.s3.amazonaws.com/images/20230216_013857_604450_dau-hieu-bi-benh-gan.max-800x800.jpg'", 'url_title': 'cac-dau-hieu-bi-benh-gan'}
{'link': 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/nhi/cac-trieu-chung-cua-benh-soi-o-tre-so-sinh/', 'majority': 'pediatrics', 'image': "'https://vinmec-prod.s3.amazonaws.com/images/20230126_023041_074220_cac-trieu-chung-cua-b.max-800x800.jpg'", 'url_title': 'cac-trieu-chung-cua-benh-soi-o-tre-so-sinh'}
{'link': 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/san-phu-khoa-va-ho-tro-sinh-san/ba-bau-bi-viem-mui-di-ung-co-sao-khong/', 'majority': 'obgyn', 'image': "'https://vinmec-prod.s3.amazonaws.com/images/20190407_080657_550995_ba-bau-bi-viem-mui-di.max-800x800.jpg'", 'url_title': 'ba-bau-bi-viem-mui-di-ung-co-sao-khong'}
{'link': 'https://www.vinmec.com/vi/ung-buou-xa-tri/thong-tin-suc-khoe

In [ ]:
print(len(crawl_url))

1363


In [ ]:
import IPython.display as display

def crawl_and_modify_links(url_list):
    for i, url_obj in enumerate(url_list):
        url = url_obj["link"]
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        block_content = soup.find('div', {'class': 'block-content'})

        try:
            # Remove class ".this_class_to_hide_all" (only remove that className, not remove the tag)
            hide_elements = block_content.select('.this_class_to_hide_all')
            for element in hide_elements:
                element.attrs['class'] = []
        except Exception as e:
            print(f"Error occurred while removing class names: {e}")

        try:
            # Remove all elements with class name "post-banner-middle"
            post_banner_middle_elements = block_content.select('.post-banner-middle')
            for element in post_banner_middle_elements:
                element.decompose()
        except Exception as e:
            print(f"Error occurred while removing elements: {e}")

        # Extract the first 'b' tag and add its content as the "snippet" field
        first_b_tag = block_content.find('b')
        snippet = first_b_tag.get_text(strip=True) if first_b_tag else None
        url_obj["snippet"] = snippet

        # Extract the tag with class name "detail-header" and add its content as the "title" field
        detail_header_tag = soup.find(class_="detail-header")
        title = detail_header_tag.get_text(strip=True) if detail_header_tag else None
        url_obj["title"] = title

        # Add the modified block content as a new field "htmlString" in the URL object
        url_obj["content"] = str(block_content)

        # Display progress
        display.clear_output(wait=True)
        display.display(display.HTML(f"<h3>Processing URL {i+1}/{len(url_list)}</h3>"))

    # Clear progress display after completion
    display.clear_output(wait=True)

    return url_list

crawl_url = crawl_and_modify_links(crawl_url)

for item in crawl_url:
    print(item)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import math
import zipfile

chunk_size = 100
num_chunks = math.ceil(len(crawl_url) / chunk_size)

zip_file_name = "crawl_url_chunks.zip"
with zipfile.ZipFile(zip_file_name, "w") as zip_file:
    for i in range(num_chunks):
        start_index = i * chunk_size
        end_index = start_index + chunk_size
        chunk = crawl_url[start_index:end_index]

        file_name = f"crawl_url_{i+1}.json"
        with open(file_name, "w") as file:
            file.write(str(chunk))

        zip_file.write(file_name)

print("Downloading zip file...")
files.download(zip_file_name)


Saved crawl_url_1.json in crawl_url_chunks.zip
Saved crawl_url_2.json in crawl_url_chunks.zip
Saved crawl_url_3.json in crawl_url_chunks.zip
Saved crawl_url_4.json in crawl_url_chunks.zip
Saved crawl_url_5.json in crawl_url_chunks.zip
Saved crawl_url_6.json in crawl_url_chunks.zip
Saved crawl_url_7.json in crawl_url_chunks.zip
Saved crawl_url_8.json in crawl_url_chunks.zip
Saved crawl_url_9.json in crawl_url_chunks.zip
Saved crawl_url_10.json in crawl_url_chunks.zip
Saved crawl_url_11.json in crawl_url_chunks.zip
Saved crawl_url_12.json in crawl_url_chunks.zip
Saved crawl_url_13.json in crawl_url_chunks.zip
Saved crawl_url_14.json in crawl_url_chunks.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
display.max_output = None

# print(crawl_url[:200])
print(crawl_url[201:400])
# print(crawl_url[401:600])
# print(crawl_url[601:800])
# print(crawl_url[801:1000])
# print(crawl_url[1001:1200])
# print(crawl_url[1201:])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

